# Perth Machine Learning Group Poem Generator

## Introduction

The following code uses GRU to generate poems. It reads through a corpus of poems, and learns sequences of characters, including line breaks and titles.

In short, it observes many sequences of characters, and infers the character that should come next. For instance, it guesses that after 'The cat eat' should come the letter 's'.

Further details will be given with the code.

## The code

### Data exploration

In [ ]:
import tensorflow as tf  # version 1.9 or above
tf.enable_eager_execution()  # Execution of code as it runs in the notebook. Normally, TensorFlow looks up the whole code before execution for efficiency.

from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras import Model
#from tensorflow.data.Dataset import from_tensor_slice
from tensorflow.train import AdamOptimizer
from tensorflow.losses import sparse_softmax_cross_entropy
import numpy as np
import re
import time

In [ ]:
path = 'corpus.txt'

In [ ]:
with open(path, encoding='utf-8') as f:
    text = f.read().lower()
text = re.sub('[^a-z\n]', ' ', text)

Now we want the text backward so as to predict the previous letters instead of the next. The idea is to feed the model with a rhyme and get a line.

In [ ]:
text = text[::-1]

### Dataset creation

In [ ]:
unique = sorted(set(text))  # unique contains all the unique characters in the corpus

char2idx = {u:i for i, u in enumerate(unique)}  # maps characters to indexes
idx2char = {i:u for i, u in enumerate(unique)}  # maps indexes to characters

In [ ]:
max_length = 100  # Maximum length sentence we want per input in the network
vocab_size = len(unique)
embedding_dim = 128  # number of 'meaningful' features to learn. Ex: ['queen', 'king', 'man', 'woman'] has a least 2 embedding dimension: royalty and gender.
units = 512  # In keras: number of output of a sequence. In short it rem
BATCH_SIZE = 64
BUFFER_SIZE = 10000

In [ ]:
input_text = []
target_text = []

for f in range(0, len(text) - max_length, max_length):
    inps = text[f : f + max_length]
    targ = text[f + 1 : f + 1 + max_length]
    input_text.append([char2idx[i] for i in inps])
    target_text.append([char2idx[t] for t in targ])

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((input_text, target_text)).shuffle(BUFFER_SIZE)
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

### Explaination

In fact, the algorithm does not learn which characters comes next. It analyzes sequences of characters as inputs (ex: 'abcd'), and predicts sequences as outputs (ex: 'bcde').

Why?

During the training phase, it learns more that just the next character. It updates weights for each characters from the input sequence to the output sequence.

> Consider the sequences 'abcd', 'bcde', 'cdef', 'defg', the letter "d" is given different weights that depend on the previous sequences

The use of these updates helps predicting better the next sequences and so on. So it learns the next character but also all the subsequent weights to better predict the next letter

In our dataset, an example of input and target are:

In [ ]:
# example of input:
print('Given the following sequence: \n\n')
print(''.join(idx2char[input_text[14][i]] for i in range(len(target_text[0]))))
print('\n\n')
print('the network has to learn that a correct continuation is: \n')
# example of output the algorithm has to learn
print(''.join(idx2char[target_text[14][i]] for i in range(len(input_text[0]))))

### Model

We build a model with:
  * an embedding layer to prepare output to feed the GRU layer
  * a GRU (Gated Recurrent Unit) layer
  * a regular neural network layer

In [ ]:
class Model(Model):
  def __init__(self, vocab_size, embedding_dim, units, batch_size):
    super(Model, self).__init__()
    self.units = units
    self.batch_sz = batch_size
    self.embedding = Embedding(vocab_size, embedding_dim)
    self.gru = GRU(self.units, 
                   return_sequences=True, 
                   return_state=True, 
                   recurrent_activation='sigmoid', 
                   recurrent_initializer='glorot_uniform')
    self.fc = Dense(vocab_size)
        
  def call(self, x, hidden):
    x = self.embedding(x)
    output, states = self.gru(x, initial_state=hidden)
    output = tf.reshape(output, (-1, output.shape[2]))
    x = self.fc(output)
    return x, states

In [ ]:
model = Model(vocab_size, embedding_dim, units, BATCH_SIZE)

Then we choose a regular AdamOptimizer, and a cross entropy loss funtion

In [ ]:
optimizer = AdamOptimizer()

In [ ]:
def loss_function(real, preds):
    return sparse_softmax_cross_entropy(labels=real, logits=preds)

### Training

We train the model over 100 epoch (you can train it longer if you want)

In [ ]:
n_epochs = 100

for epoch in range(n_epochs):
    start = time.time()
    hidden = model.reset_states()  # initializes the hidden state at the start of every epoch
    
    for (batch, (inp, target)) in enumerate(dataset):
          with tf.GradientTape() as tape:
              predictions, hidden = model(inp, hidden)  # predicts next letter given an input
              target = tf.reshape(target, (-1, ))
              loss = loss_function(target, predictions)  # compares the prediction with the real output

          grads = tape.gradient(loss, model.variables)
          optimizer.apply_gradients(zip(grads, model.variables), global_step=tf.train.get_or_create_global_step())  # Gradient descent

          if batch % 100 == 0:
              print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, loss))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch + 1, loss))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

### Save our model

We often want to save a model and use it later. Here is a way to do it. (some improvements can be made)

First, put all parameters (hyperparameters and weights) in a dictionary

In [ ]:
parameters = {}
parameters['max_length'] = max_length
parameters['vocab_size'] = vocab_size
parameters['embedding_dim'] = embedding_dim
parameters['units'] = units
parameters['BATCH_SIZE'] = BATCH_SIZE
parameters['BUFFER_SIZE'] = BUFFER_SIZE
parameters['embedding_weights'] = embedding_weights
parameters['gru_weights'] = gru_weights
parameters['fc_weights'] = fc_weights
parameters['char2idx'] = char2idx
parameters['idx2char'] = idx2char

Second, save the parameters, the weights for every layers we have trained (embedding, gru, fc) in a npy file

In [ ]:
np.save('model_poems', parameters)

### Text Generation

We can now see how the model performs. 

A value we want to tune is the temperature. It tells how 'random' we want our predictions to be. The lowest the value, the more the prediction is random; giving nonsense. Greater values favorize the letter that is the most probable; creating a lot of repetitions.

Feel free to change the rhymes and the temperatures.

In [ ]:
rhymes = [' the child', ' be', ' me', ' wild']

# We test different temperatures
for temperature in [0.2, 0.5, 0.8, 0.9, 1.1]:
    print('Temperature = {} \n'.format(temperature))
    text_generated = ''
    
    # Here we generate a new line given the last word (the rhyme)
    for rhyme in rhymes:
        num_generate = 150  # maximum number of characters to generate per line
        start_string = text_generated + rhyme[::-1]  # remember we generate poems from the end to the start
        input_eval = [char2idx[s] for s in start_string]  # converts start_string to numbers the model understands
        input_eval = tf.expand_dims(input_eval, 0)  # 
        hidden = [tf.zeros((1, units))]  # initializes the hidden layer to zeros (there's maybe a better approach)
        
        b = True
        c = 1
        text_to_add = ''
        tmp = start_string + ' '
        while b == True:
            # Add predicted letters
            predictions, hidden = model(input_eval, hidden)  # predictions holds the probabily for each character to be most adequate continuation
            predictions = predictions / temperature  # alters characters' probabilities to be picked (but keeps the order)
            predicted_id = tf.multinomial(tf.exp(predictions), num_samples=1)[0][0].numpy()  # picks the next character for the generated text
            input_eval = tf.expand_dims([predicted_id], 0)
            tmp += idx2char[predicted_id]
            text_to_add += idx2char[predicted_id]
            
            c += 1
            
            # Stop as soon as there is a line break
            if idx2char[predicted_id] == '\n' or c > num_generate:
                text_generated += rhyme[::-1] + text_to_add 
                b = False
    print('Poem : \n')
    print (text_generated[::-1])

## Conclusion

That's promising:
* It can spell words correctly
* There is some structure (line breaks).

Harder-to-fix issue: Lines make little sense.

Possible improvements:
* further training
* hyperparameters tuning
* more consistent corpus (some word are truncated with a bit a slang)

*Note: Punctunation was removed*